In [ ]:
!pip install -q --upgrade bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.8 MB/s eta 0:00:00


In [ ]:
# -----------------------------------------
# INSTALL REQUIRED LIBRARIES (Google Colab)
# -----------------------------------------
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft

# -----------------------------------------
# IMPORT LIBRARIES
# -----------------------------------------
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
import hnswlib
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
    BitsAndBytesConfig,
)
from peft import get_peft_model, LoraConfig, TaskType
import torch
import os

# -----------------------------------------
# MOUNT GOOGLE DRIVE
# -----------------------------------------
drive.mount('/content/drive')

# -----------------------------------------
# STEP 1: UPLOAD AND LOAD DATASET
# -----------------------------------------
from google.colab import files
uploaded = files.upload()  # Upload ARS_DATA.csv manually
df = pd.read_csv("ARS_DATA.csv")

# -----------------------------------------
# STEP 2: CLEANING AND ENCODING
# -----------------------------------------
def clean_and_encode_data(df):
    df.dropna(inplace=True)
    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]  # Store original values for later
        df[col] = encoders[col].fit_transform(df[col])
    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])
    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)

# -----------------------------------------
# STEP 3: CREATE HUMAN-READABLE DESCRIPTIONS (USE ORIGINAL VALUES)
# -----------------------------------------
def create_description(df):
    df['Full_Description'] = df.apply(lambda row:
        f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
        f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
        f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
        axis=1)
    return df

df = create_description(df)

# -----------------------------------------
# STEP 4: GENERATE EMBEDDINGS
# -----------------------------------------
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

# -----------------------------------------
# STEP 5: BUILD HNSW INDEX
# -----------------------------------------
def build_hnsw_index(embeddings, ef=100, M=16):
    dim = embeddings.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=ef, M=M)
    index.add_items(embeddings, ids=list(range(len(embeddings))))
    index.set_ef(ef)
    return index

index = build_hnsw_index(embeddings)

# -----------------------------------------
# STEP 6: PREPARE FOR GPT FINE-TUNING
# -----------------------------------------
def prepare_finetuning_data(df):
    df['input'] = df['Full_Description'].apply(lambda x: f"{x} => Recommended meal and workout:")
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

ds = prepare_finetuning_data(df)

# -----------------------------------------
# STEP 7: LOAD MODEL + APPLY LoRA ADAPTERS (Required for 8-bit fine-tuning)
# -----------------------------------------
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Load 8-bit quantized base model
quant_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

# Add LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# -----------------------------------------
# STEP 8: TOKENIZE DATA
# -----------------------------------------
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = ds.map(tokenize_function, batched=True)
small_ds = tokenized_dataset.select(range(min(7000, len(tokenized_dataset))))

# -----------------------------------------
# STEP 9: TRAINING ARGUMENTS & TRAINER
# -----------------------------------------
save_dir = "/content/drive/MyDrive/deepseek"
os.makedirs(save_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=save_dir,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir=os.path.join(save_dir, 'logs'),
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -----------------------------------------
# STEP 10: TRAIN & SAVE MODEL + ARTIFACTS TO GOOGLE DRIVE
# -----------------------------------------
trainer.train()

model.save_pretrained(save_dir)           # Save LoRA weights and model
tokenizer.save_pretrained(save_dir)       # Save tokenizer

df.to_csv(os.path.join(save_dir, "ARS_df.csv"), index=False)          # Save DataFrame
np.save(os.path.join(save_dir, "ARS_embeddings.npy"), embeddings)    # Save embeddings
index.save_index(os.path.join(save_dir, "ARS_hnsw_index.bin"))       # Save HNSW index

print(f"All artifacts saved to {save_dir}")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s

Saving ARS_DATA.csv to ARS_DATA.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

<ipython-input-1-48e4181caa83>:158: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


All artifacts saved to /content/drive/MyDrive/deepseek


In [ ]:
# -----------------------------------------
# INSTALL REQUIRED LIBRARIES (Google Colab)
# -----------------------------------------
!pip install -q transformers sentence-transformers hnswlib

# -----------------------------------------
# IMPORT AND MOUNT DRIVE
# -----------------------------------------
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# -----------------------------------------
# LOAD SAVED ARTIFACTS FROM DRIVE
# -----------------------------------------
base_path = "/content/drive/MyDrive/deepseek"

df = pd.read_csv(f"{base_path}/ARS_df.csv")
embeddings = np.load(f"{base_path}/ARS_embeddings.npy")

dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index(f"{base_path}/ARS_hnsw_index.bin")
index.set_ef(100)

tokenizer = AutoTokenizer.from_pretrained(base_path)
model = AutoModelForCausalLM.from_pretrained(base_path).to("cuda")
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# -----------------------------------------
# RECOMMEND FUNCTION
# -----------------------------------------
def recommend(query, embedder, index, df, gen_pipeline, top_k=3):
    query_vec = embedder.encode([query])
    ids, distances = index.knn_query(query_vec, k=top_k)
    similar_profiles = df.iloc[ids[0]]['Full_Description'].tolist()
    context = "\n".join(similar_profiles)
    prompt = f"Based on similar users:\n\n{query} => Recommended meal and workout:"
    response = gen_pipeline(prompt, max_new_tokens=50, do_sample=True)[0]['generated_text']
    return response

# -----------------------------------------
# EXAMPLE QUERY
# -----------------------------------------
query = "Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
response = recommend(query, embedder, index, df, gen_pipeline)
print("\n--- Recommendation Output ---\n")
print(response)



--- Recommendation Output ---

Based on similar users:

Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout: Keto Omelette (2x/week), Light Protein (2x/week)

Another user: 

Female, 27 years old, 170 cm, 65 kg, Protein Split (High Pro) (2x


In [ ]:
query = "Male, 47 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
response = recommend(query, embedder, index, df, gen_pipeline)
print("\n--- Recommendation Output ---\n")
print(response)


--- Recommendation Output ---

Based on similar users:

Male, 47 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout: Keto Omelette (2x/week), Protein Smoothie (2x/week), High Pro veggies (1x/week)

Another user: 
Male, 29 years old, 150 cm, 75 kg
